In [8]:
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')
# 现在可以安全地使用环境变量了
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# 开启langsmith服务
#os.environ['LANGCHAIN_TRACING_V2'] = "true"
#os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
#os.environ['LANGCHAIN_API_KEY']="ls__8f359b12028e4a4bbfb32616d6158173"
#os.environ['LANGCHAIN_PROJECT'] = "My Project"
from paperqa_ import Docs
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
from langchain.chat_models import ChatOpenAI

In [2]:
class _docs_maker():
    _BasePath_ = ""
    docs:Docs =None 
    def __init__(self,base_path:str = None) -> None:
        if base_path:
            self._BasePath_ = base_path
    
    def create_new_doc(self):
        my_docs_path = []  
        my_docs_citation = []   
        my_docs_doc_key = []
        my_docs_doc_name = []
        my_docs_doc_abstrst = []
        from tqdm import tqdm
        pdf_path = f"{self._BasePath_}/literature_PDF"
        df = pd.read_csv(f"{self._BasePath_}\litera_lis.csv",encoding="ISO-8859-1",index_col=0)
        for _x in tqdm(range(df.shape[0])):  # list(df.index):
            # info = df.loc[_x] 
            is_down = df['is_download'].iloc[_x]
            if is_down ==1 :
                _key = df.index[_x].replace("SL01",f"{_f}")
                my_docs_path.append(f"{pdf_path}/{df.index[_x]}.pdf")
                my_docs_citation.append(df['Citation'].iloc[_x])
                my_docs_doc_key.append(f"{_key}")
                my_docs_doc_name.append(f"{df['Title'].iloc[_x]}")
                my_docs_doc_abstrst.append(f"{df['Abstract'].iloc[_x]}")
        docs = Docs()
        error_litera = []
        for i,d in tqdm(enumerate(my_docs_path)):
            try:
                docs.add(path=d,
                        citation=my_docs_citation[i],
                        dockey=my_docs_doc_key[i],
                        docname=my_docs_doc_name[i],
                        abstract=my_docs_doc_abstrst[i])
            except:
                error_litera.append(d)
        from langchain.chat_models import ChatOpenAI
        llm = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")
        llm_4 = ChatOpenAI(temperature=0.5, model="gpt-4")
        docs.update_llm(llm=llm_4,summary_llm=llm)
        self.docs = docs

    def save_docs(self,path:str):
        # save
        import pickle
        with open(path, "wb") as f:
            pickle.dump(self.docs, f)
    def load_docs(self,path:str):
        from pathlib import Path
        import pickle
        pkl_path = Path(path)
        with pkl_path.open("rb") as f:
            docs = pickle.load(f)
        self.docs = docs

In [3]:
# 加载问答内核
docs_maker = _docs_maker()
docs_maker.load_docs("docs_V04.pkl")

In [33]:
# 保存内核
#docs_maker.save_docs(path="docs_V04.pkl")

In [4]:
docs_maker.docs.query("你是谁")

c:\Users\donkey\.conda\envs\PK_LLM\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `arun` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use ainvoke instead.
  warn_deprecated(


Answer(question='你是谁', answer='I cannot answer.', context='Hong, Xiaowen, et al. "PFASs in Cerebrospinal Fluids and Blood−CSF Barrier Permeability in Patients with Cognitive Impairment." Environmental Science & Technology, vol. 58, no. 9, 2024, pp. 5129-5138.: - Study focused on PFASs in cerebrospinal fluids (CSF) and blood, examining their transfer and cognitive implications.\n- Identified key transporter proteins involved: P-glycoprotein (P-gp), BCRP, MRPs, and OATs.\n- Age was a significant factor in PFAS transfer, with a notable increase in RCSF/Serum values after age 60.\n- Logistic regression analysis linked CSFPFOS levels to increased cognitive impairment risk.\n- CSFPFOA showed a protective effect on cognitive impairment.\n- Toxicity Quotient (TQ) analysis indicated PFOS in CSF may have a greater potential to impair cognitive function than PFOA and PFHxS.\n- Score: 9\n\nValid keys: PFASs in Cerebrospinal Fluids and Blood-CSF Barrier Permeability in Patients with Cognitive Impai

In [7]:
docs_maker.docs.doc_index.save_local("_data/demo_index")

In [8]:
docs_maker.docs.__getstate__


KeyboardInterrupt



In [13]:
# 进行提示词工程
from langchain.prompts import PromptTemplate
summary_prompt = PromptTemplate(
    input_variables=["text", "citation", "question", "summary_length"],
    template="""
Objective:
* To understand and distill the provided text, extracting key information that specifically addresses the posed question.
* The provided text comes from a literature excerpt, where useful information may involve the background,\
    objectives, methods, and conclusions of the study.

Instructions:
* Thoroughly read and comprehend the text.
* Identify and extract information of the text that are most relevant to the specified question.
* Structure the information using bullet points or a concise paragraph to enhance readability.
* Emphasize the critical points that directly respond to or elucidate the posed question.

Exclusions:
* Exclude any information not directly pertinent to the question to maintain focus and relevance.

Output Format:
* Prepare the information to function effectively as input for another LLM, \
    facilitating subsequent analysis or response generation.

Clarity and Precision:
* Ensure the information is explicitly clear and precise to eliminate ambiguities, \
    focusing on usability for further language model processing.
    
"""
    'Reply "Not applicable" if text is irrelevant. '
    "Use {summary_length}. At the end of your response, provide a score from 1-10 on a newline "
    "indicating relevance to question. Do not explain your score. "
    "\n\n"
    "{text}\n\n"
    "Excerpt from {citation}\n"
    "Question: {question}\n"
    "Relevant Information Summary:",
)
docs.prompts.summary = summary_prompt

In [19]:
q= "如何测试PFAS在人乳中的含量"
llm = ChatOpenAI(temperature=0)
a= llm.invoke(f"将这段话翻译为英文:{q}")
r  = docs.query(q,length_prompt='as long as you can')
b= llm.invoke(f"将这段话翻译为中文:{r.formatted_answer}")
print(b.content)
print(r.references)

问题：如何测试PFAS在人乳中的含量

测试PFAS（全氟烷基化合物）在人乳中的含量需要进行多步实验操作。首先，人乳样本需要经过提取和清洁，这一步骤包括将5毫升的人乳样本加入离心管中，然后加入内标溶液，并与甲酸进行涡旋混合。混合物随后转移到固相萃取柱，进行冲洗，真空干燥，并用甲醇洗脱以供分析。

PFAS的分析通常使用高效液相色谱仪（HPLC）与三重四极杆质谱仪（TOF MS）进行，具体的仪器参数需要详细说明。色谱分离使用Kinetex®C18 1.7µm: 2.1 ×100 mm柱，并使用10 mM醋酸铵在水和甲醇中的梯度。PFAS的质谱鉴定使用PerkinElmer®QSight ™220三重四极杆质谱仪，在负电喷雾电离模式下进行。

对于PFASs的定量，还涉及到PFOA，PFNA，PFDA，全氟十一酸，全氟十二酸的酯化，使用氯甲酸酯使它们形成半挥发性的甲酯。气相色谱（GC）分析的衍生化方法包括二氮甲烷，溴化苄在液液萃取（LLE）期间，2,4-二氟苯胺，强阴离子交换萃取与碘甲烷，甲醇或丁醇。

最后，人乳中单个PFAS的定量是通过使用MS/MS转换和保留时间进行的，这些信息在表SD-1中有报告。基于Thompson等人（2010）的一阶药代动力学（PK）模型被用来预测支持人乳中测量到的PFAS浓度所需的暴露水平。
1. Chemosphere. 2020 May;246:125724. doi: 10.1016/j.chemosphere.2019.125724. Epub 2019 Dec 20.:(Concentrations of perfluoroalkyl substances in human milk from Ireland: Implications for adult and nursing infant exposure pages 10-11) 

2. Chemosphere. 2020 May;246:125724. doi: 10.1016/j.chemosphere.2019.125724. Epub 2019 Dec 20.:(Concentrations of perfluoroalkyl substances in human milk from Ireland: Implications for adult and nu

In [20]:
print(r.contexts[0].context)

- Breast milk samples were extracted and cleaned up following methods by .
- Besides PFOS, PFOA, PFNA, and PFHxS, other PFASs measured included PFBS, FOSA, MeFOSA, EtFOSA, MeFOSE, and EtFOSE.
- Sample preparation involved adding 5 mL of breast milk to a centrifuge tube, spiking it with an internal standard solution, and vortexing with formic acid.
- The mixture was transferred to a solid phase extraction cartridge, rinsed, dried under vacuum, and eluted with MeOH for analysis.
- PFAS analysis was conducted using an HPLC coupled to a triple TOF MS, with specific instrumental parameters detailed.

Relevance Score: 10


In [18]:
q= "有哪些处理PFAS污染，请你撰写一段总结文字"
llm = ChatOpenAI(temperature=0)
a= llm.invoke(f"将这段话翻译为英文:{q}")
r  = docs.query(q,length_prompt='as long as you can')
b= llm.invoke(f"将这段话翻译为中文:{r.formatted_answer}")
print(b.content)
print(r.references)

处理PFAS（全氟和多氟烷基物质）污染的方法包括各种样品预处理方法，如蛋白质沉淀、水解、液液萃取（LLE）以及固相萃取（SPE）。在SPE中，使用了不同的柱子，包括ACQUITY BEH C18、Gemini C18、Waters Acquity UPLC BEH C18、Discovery HS C18、Waters Acquity UPLC HSS T3、J &W DB-5MS和J &W DB-35MS等。此外，还使用了各种不同的分析技术，如HPLC-ESI-MS/MS、LC-ESI-HRMS、GC-EI-MS和UHPLC-ESI-MS/MS等。在处理PFAS污染时，回收率的范围通常在61%到160%之间，具体取决于所研究的PFAS化合物。此外，液相色谱-串联质谱（LC-MS/MS）在多反应监测（MRM）模式下是量化PFAS的首选方法。
1. Environ Sci Pollut Res Int. 2018 Dec;25(36):36064-36086. doi: 10.1007/s11356-018-3483-z. Epub 2018 Nov 1.:(Per- and polyfluoroalkyl substances in human breast milk and current analytical methods pages 14-16) 

2. J Chromatogr A. 2011 Jul 15;1218(28):4312-21. doi: 10.1016/j.chroma.2011.04.061. Epub 2011 May 7.:(Simple, high throughput ultra-high performance liquid chromatography/tandem mass spectrometry trace analysis of perfluorinated alkylated substances in food of animal origin: milk and fish pages 1-2) 

3. Chemosphere. 2020 May;246:125724. doi: 10.1016/j.chemosphere.2019.125724. Epub 2019 Dec 20.:(Concentratio

***

In [17]:
answer = docs.query("How does the PHT1;8 protein influence the translocation of PFOS in plants?")
print(answer.formatted_answer)

Question: How does the PHT1;8 protein influence the translocation of PFOS in plants?

I cannot answer.

References

1. Chemosphere. 2020 May;246:125724. doi: 10.1016/j.chemosphere.2019.125724. Epub 2019 Dec 20.:(Concentrations of perfluoroalkyl substances in human milk from Ireland: Implications for adult and nursing infant exposure pages 17-18) 

2. Environ Sci Pollut Res Int. 2018 Dec;25(36):36064-36086. doi: 10.1007/s11356-018-3483-z. Epub 2018 Nov 1.:(Per- and polyfluoroalkyl substances in human breast milk and current analytical methods pages 9-9) 

3. Environ Sci Pollut Res Int. 2018 Dec;25(36):36064-36086. doi: 10.1007/s11356-018-3483-z. Epub 2018 Nov 1.:(Per- and polyfluoroalkyl substances in human breast milk and current analytical methods pages 16-17) 



In [ ]:
# load
import pickle
from pathlib import Path
pkl_path = Path("docs_V02.pkl")
with pkl_path.open("rb") as f:
    docs = pickle.load(f)

In [ ]:
a = docs.query(
    query = 'Which PFASs were found to be major contributors and which ones showed high detection frequencies?',
    k=5,
    key_filter = False
    )

In [ ]:
a.dockey_filter

In [ ]:
s = """
Perfluoroalkyl and polyfluoroalkyl substances (PFAS) are potentially related to many adverse health outcomes and could be transferred from maternal blood to human milk, which is an important exposure source for infants during a long-term period. In this study, the maternal blood of 76 women after delivery and their matched human milk samples obtained at 0.5, 1, and 3 months were analyzed by solid-phase extraction method with metal?organic framework/polymer hybrid nanofibers as the sorbents and ultrahigh-performance liquid chromatography-negative electrospray ionization mass spectrometric for quantitative analysis of 31 PFAS. The perfluorooctanoic acid, perfluorooctane sulfonate, and N-methyl perfluorooctane sulfonamido acetic acid (N-MeFOSAA) contributed to more than approximately 50% of the total PFAS concentrations in blood and human milk, while N-MeFOSAA (median: 0.274 ng/mL) was the highest PFAS in human milk at 3 months. The transfer efficiencies for PFAS from maternal blood to human milk at 0.5 months were generally lower, with medians ranging from 0.20% to 16.9%. The number of PFAS species detected in human milk increased as the lactation time went on from 0.5 to 3 months, and the concentrations of 10 PFAS displayed an increasing trend as the prolongation of lactation time (p < 0.05). ? 2024 American Chemical Society.
"""

In [ ]:
b = docs.query(
    query = f'Which PFAS contributed to more than approximately 50% of the total PFAS concentrations in blood and human milk? \n\n{s}',
    k=5,
    #key_filter = True
    )

In [ ]:
print(b.formatted_answer)



In [ ]:
print(a.references)